In [1]:
pip install tflearn

     |████████████████████████████████| 107 kB 6.5 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=f8bda6dc600df9e7a01de089f05ea90058c72b66812853964f8cc1b41d214076
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn


In [2]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
import tflearn
import tensorflow
import random
import json
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
with open("intents.json") as file:
    data = json.load(file)

try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)


    training = numpy.array(training)
    output = numpy.array(output)

    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)



In [4]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)

In [5]:
tensorflow.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

model.fit(training, output, n_epoch=1500, batch_size=8, show_metric=True)
model.save("model.tflearn")

Training Step: 6599  | total loss: 0.11051 | time: 0.165s
| Adam | epoch: 200 | loss: 0.11051 - acc: 0.9756 -- iter: 256/263
Training Step: 6600  | total loss: 0.10152 | time: 0.175s
| Adam | epoch: 200 | loss: 0.10152 - acc: 0.9780 -- iter: 263/263
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [5]:
user_name = input()
bot_name = "JOY"
bot_template =  bot_name + " : "
user_template = user_name + " : "

Priyanka


In [7]:
def chat():
  print("Hi, this is JOY, nice to see you")
  while True:
    inp = input(user_template)
    if inp.capitalize() == "Stop":
      break

    results = model.predict([bag_of_words(inp, words)])
    results_index = numpy.argmax(results)
    tag = labels[results_index]
      
    for tg in data["intents"]:
      if tg['tag'] == tag:
        responses = tg['responses']
    
    print(random.choice(responses))
   

chat()

Hi, this is JOY, nice to see you
Pri : hi
Hey
Pri : How are you
I am good, thank you
Pri : I am sad
Can't see you like this, please tell me what's wrong?
Pri : My parents don't love me
Hey, parents are after all parents. They just pretend to hate us but the truth is, no one can love us like them. Time will heal everything. Stay positive
Pri : I am broken
Hey, calm down, I don't know why it happened but I just know that everything happens for a reason, everything will be fine soon
Pri : Thank you
Always!
Pri : I am bad
Not at all dear. I think you are very sensitive. And sensitive people get hurt very often and that doesn't make you bad or wrong. Trust yourself. You are the best.
Pri : stop
